In [1]:
import pyforest
from rdkit import Chem
from rdkit.Chem import AllChem
from imblearn.over_sampling import SMOTEN, ADASYN, SMOTE 

In [3]:
datapath = '/Users/jayceepang/msse/ATOM_CODE/datasets/holdout_data/'
rf_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/RF_results/'
gp_path ='/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/results/GP_results/'

In [77]:
df = pd.read_csv(datapath+'final_pIC50s_for_ML_noOut_noDupes_pivot.csv')
df = df.rename(columns={'smiles': 'base_rdkit_smiles'})
print(df.shape)
df=df.dropna()
print(df.shape)
# df.to_csv(datapath+'final_pIC50s_for_ML_noOut_noDupes_pivot.csv', index=False)
display(df)

<IPython.core.display.Javascript object>

(400, 10)
(400, 10)


,Compound,base_rdkit_smiles,NEK1,NEK11,NEK2,NEK3,NEK4,NEK5,NEK6,NEK9
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,4.522879,4.741265,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,4.522879,4.522879,4.536830,4.522879,4.543123,4.522879,4.522879,4.522879
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,4.522879,4.772799,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879,4.522879
...,...,...,...,...,...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,5.338173,4.522879,4.522879,6.231165,4.522879,6.334277,4.522879,5.329514
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,4.956505,4.701115,4.990639,4.522879,4.746844,6.067683,4.522879,5.269689
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,5.021631,4.987176,4.920219,4.522879,5.030679,4.522879,4.522879,4.771226
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,4.522879,4.522879,4.522879,4.522879,4.522879,4.880795,4.522879,5.908011


In [72]:
df['smiles'].unique().shape

(400,)

In [33]:
og_datapath = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/paper/datasets/original_datasets/'
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition']
all_smiles_lists = []
for nek in [2, 3, 5, 9]: 
    nek_type_list = ['binding']
    if nek in [2, 9]: 
        nek_type_list =['binding', 'inhibition']
    for bi in nek_type_list: 
        nek_df = pd.read_csv(f'{og_datapath}NEK{nek}_1_uM_min_50_pct_{bi}.csv')
        all_smiles_lists.append(nek_df['base_rdkit_smiles'].values)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
all_smiles = []
for sublist in all_smiles_lists:
    all_smiles.extend(sublist)
len(all_smiles)

In [79]:
mol_in_og_data = df[df['base_rdkit_smiles'].isin(all_smiles)]
new_df = df[~df['base_rdkit_smiles'].isin(all_smiles)]
mol_in_og_data
new_df
print(f'new nanobret data size after filtering: {new_df.shape}')
for nek in [ x for x in new_df.columns if x.startswith('NEK')]: 
    new_df[f'{nek}_active']=new_df[nek].apply(lambda x: 1 if x>6 else 0)

new nanobret data size after filtering: (400, 10)


In [84]:
for nek in [ x for x in new_df.columns if x.endswith('active')]: 
    nek_str = nek.split('_')[0]

    cols = ['Compound', 'base_rdkit_smiles', nek] 
    nek_df = new_df[cols]
    # display(nek_df)
    display(nek_df[nek].value_counts())
    nek_df=nek_df.rename(columns={'Compound':'compound_id', f'{nek_str}_active': 'active'})
    nek_df['NEK'] = nek_str
    nek_df['subset']='test'
    display(nek_df)
    nek_df.to_csv(f'{datapath}{nek_str}_uM_min_50_pct.csv', index=False)
    

NEK1_active
0    370
1     30
Name: count, dtype: int64

,compound_id,base_rdkit_smiles,active,NEK,subset
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,0,NEK1,test
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,0,NEK1,test
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,0,NEK1,test
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,0,NEK1,test
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,0,NEK1,test
...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,0,NEK1,test
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,0,NEK1,test
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,0,NEK1,test
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,0,NEK1,test


NEK11_active
0    398
1      2
Name: count, dtype: int64

,compound_id,base_rdkit_smiles,active,NEK,subset
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,0,NEK11,test
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,0,NEK11,test
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,0,NEK11,test
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,0,NEK11,test
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,0,NEK11,test
...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,0,NEK11,test
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,0,NEK11,test
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,0,NEK11,test
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,0,NEK11,test


NEK2_active
0    369
1     31
Name: count, dtype: int64

...

,compound_id,base_rdkit_smiles,active,NEK,subset
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,0,NEK5,test
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,0,NEK5,test
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,0,NEK5,test
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,0,NEK5,test
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,0,NEK5,test
...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,1,NEK5,test
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,1,NEK5,test
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,0,NEK5,test
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,0,NEK5,test


NEK6_active
0    400
Name: count, dtype: int64

,compound_id,base_rdkit_smiles,active,NEK,subset
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,0,NEK6,test
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,0,NEK6,test
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,0,NEK6,test
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,0,NEK6,test
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,0,NEK6,test
...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,0,NEK6,test
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,0,NEK6,test
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,0,NEK6,test
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,0,NEK6,test


NEK9_active
0    366
1     34
Name: count, dtype: int64

,compound_id,base_rdkit_smiles,active,NEK,subset
0,3_IN_PP1,NC1=C2C(N(C(C)(C)C)N=C2C3=CNC4=C3C=CC=C4)=NC=N1,0,NEK9,test
1,ALK_kinase_inhibitor_1,CN(CC1)CCN1C2=CC=C(NC3=NC=C4C(C(C5=C(OC)C=CC(F...,0,NEK9,test
2,ALW_II_49_7,O=C(C1=CC(NC2=CC(C(NC3=CC=CC(C(F)(F)F)=C3)=O)=...,0,NEK9,test
3,AMG_47a,O=C(NC1=CC=CC(C(F)(F)F)=C1)C2=CC=C(C)C(C3=CC4=...,0,NEK9,test
4,AST487,O=C(NC1=CC=C(OC2=NC=NC(NC)=C2)C=C1)NC3=CC=C(CN...,0,NEK9,test
...,...,...,...,...,...
395,ZDG_7_53_1,CN1CCN(C2=CC=CC(NC3=NC=CC(C4=CC=C(NC([C@@H]5CC...,0,NEK9,test
396,ZDG_7_9,O=C1N(C2CCCC2)C3=C(C=NC(NC4=CC=C(C(NCC(F)F)=O)...,0,NEK9,test
397,Zanzalintinib,O=C(C1=CC2=C(OC3=CC=C(NC(C4(CC4)C(NC5=CC=C(C=C...,0,NEK9,test
398,ba_03_55_11,FC(F)(F)C1=CC(NC2=NC(C3=C4C=CC=NN4N=C3)=CC=N2)...,0,NEK9,test


# Next steps
1. figure out binding or inhibition data --> pIC50 of 6 is equivalent to 50% inhibition at 1uM
2. how should we convert the labels to active/inactive? (our previous datasets were perecent binding/inhibition)
4. run this new data on our production models 